# Дашборд для Яндекс.Дзен

## Описание проекта

Я - аналитик в Яндекс.Дзене. Почти всё мое время занимает анализ пользовательского взаимодействия с карточками статей.  
Каждую карточку определяют её тема и источник (у него тоже есть тема). Примеры тем: «Красота и здоровье», «Россия», «Путешествия».  
Пользователей системы характеризует возрастная категория. Например, «26-30» или «45+».  

Есть три способа взаимодействия пользователей с системой:
* Карточка отображена для пользователя (show);
* Пользователь кликнул на карточку (click);
* Пользователь просмотрел статью карточки (view).

Каждую неделю менеджеры задают мне одни и те же вопросы:
* Cколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
* Как много событий генерируют источники с разными темами?
* Как соотносятся темы карточек и темы источников?

Я пришла к выводу, что процесс пора автоматизировать - сделать дашбор. Обсудив с менеджерами его состав, внешний вид и набор отображаемых данных и пообщавшись с администраторами БД, было разработано краткое ТЗ.

## Техническое задание
Бизнес-задача: анализ взаимодействия пользователей с карточками Яндекс.Дзен;   
Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю;  
Кто будет основным пользователем дашборда: менеджеры по анализу контента;  
Важность: все графики имеют равную важность;  
Источники данных для дашборда: cырые данные о событиях взаимодействия пользователей с карточками (таблица log_raw);  
База данных, в которой будут храниться агрегированные данные: дополнительные агрегированные таблицы в БД zen;   
Частота обновления данных: один раз в сутки, в полночь по UTC;  
Какие графики должны отображаться и в каком порядке, какие элементы управления должны быть на дашборде: разработан макет дашборда;

**Состав данных для дашборда:**  
- История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);  
- Разбивка событий по темам источников;  
- Таблица соответствия тем источников темам карточек;

**По каким параметрам данные должны группироваться:**  
- Дата и время;  
- Тема карточки;  
- Тема источника;  
- Возрастная группа; 

**Характер данных:**  
- История событий по темам карточек — абсолютные величины с разбивкой по минутам;
- Разбивка событий по темам источников — относительные величины (% событий);
- Соответствия тем источников темам карточек - абсолютные величины; 

## Для создания дашборда выполним следующие шаги:
* Создадим коннекцию к базе данных;
* Выполним SQL-запрос;
* Напишем код для выгрузки данных из базы данных в файл;
* В Tableau Public на основе полученного файла сформируем дашборд в соответствии с макетом.

В завершении дадим ответы на вопросы менеджеров, используя дашборд и подготовив небольшую презентацию с отчётом.

In [1]:
# Установим необходимый пакет psycopg2-binary
#!pip install psycopg2-binary

In [3]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

# Импортируем необходимые библиотеки
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# Зададим параметры подключения к БД
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

# Задаём соединение с БД
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])
# Подключаемся к БД
engine = create_engine(connection_string) 

# Сформируем текст SQL-запроса
query = ''' SELECT *
            FROM dash_visits
        '''

# Выполним SQL-запрос
dash_visits = pd.io.sql.read_sql(query, con = engine)

# Выгрузим данных из БД dash_visits в файл dash_visits.csv
dash_visits.to_csv('dash_visits.csv', sep='\t')

# Выведем случайные 5 строк файл dash_visits
dash_visits.sample(5)

,record_id,item_topic,source_topic,age_segment,dt,visits
1356,1041953,Дети,Дети,31-35,2019-09-24 18:56:00,21
5916,1046513,Знаменитости,Здоровье,18-25,2019-09-24 18:34:00,1
5040,1045637,Здоровье,Кино,31-35,2019-09-24 18:59:00,3
15067,1055664,Общество,Искусство,36-40,2019-09-24 18:56:00,8
5180,1045777,Здоровье,Полезные советы,31-35,2019-09-24 18:56:00,57


## Ссылка на дашборд на сайте Tableau Public
<https://public.tableau.com/profile/yulia7725#!/vizhome/Zen_YandexDashboard/Zen_YandexDashboard>

## Презентация - отчет для менеджеров
https://yadi.sk/i/FXg6gQDp2Z0PAA